In [15]:
from datasets import Dataset

train = Dataset.load_from_disk('train_mlc/dataset/train')
test = Dataset.load_from_disk('train_mlc/dataset/test')

In [4]:
import torch
import numpy as np

# Function to generate embeddings for a query
def embed_query(query):
    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

# Function to search the FAISS index and retrieve metadata
def search_faiss(index, query_embedding, metadata, k=5):
    query_embedding = np.array([query_embedding]).astype('float32')
    distances, indices = index.search(query_embedding, k)
    retrieved_metadata = [metadata[idx] for idx in indices[0]]
    return distances, indices, retrieved_metadata


from transformers import AutoModel, AutoTokenizer

model_name = 'DeepPavlov/rubert-base-cased-sentence'  # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
import json
import _pickle as pickle

with open('telegram_bot/metadata.json', 'r') as f:
    metadata = json.load(f)

with open('telegram_bot/all_chunks.pkl', 'rb') as f:
    all_chunks = pickle.load(f)

with open('telegram_bot/all_metadata.pkl', 'rb') as f:
    all_metadata = pickle.load(f)

import faiss

index = faiss.read_index('telegram_bot/faiss_index.index')

In [41]:
def formatting(documents):
    output = ""
    for i, doc in enumerate(documents[0]):
        output += f"Документ {i}: {doc}\n\n"
    return output

In [42]:
from tqdm import tqdm

retrieved_information = []

for sample in tqdm(train):
    query_embedding = embed_query(sample['question'])
    k = 5
    distances, indices, retrieved_metadata = search_faiss(index, query_embedding, all_metadata, k)
    retrieved_information.append(formatting(np.array(all_chunks)[indices]))
    # break

100% 666/666 [00:18<00:00, 35.22it/s]


In [44]:
train = train.add_column('retrieved', retrieved_information)

In [45]:
from tqdm import tqdm

test_retrieved_information = []

for sample in tqdm(test):
    query_embedding = embed_query(sample['question'])
    k = 5
    distances, indices, retrieved_metadata = search_faiss(index, query_embedding, all_metadata, k)
    test_retrieved_information.append(formatting(np.array(all_chunks)[indices]))
    # break

100% 167/167 [00:04<00:00, 37.10it/s]


In [47]:
test = test.add_column('retrieved', test_retrieved_information)

In [50]:
train.save_to_disk('train_mlc/rag_dataset/train')

Saving the dataset (0/1 shards):   0%|          | 0/666 [00:00<?, ? examples/s]

In [51]:
test.save_to_disk('train_mlc/rag_dataset/test')

Saving the dataset (0/1 shards):   0%|          | 0/167 [00:00<?, ? examples/s]